In [ ]:
#default_exp web.auth

# Web Auth

> Handles user registration and authentication.

In [ ]:
#export
import functools

from flask import Blueprint
from flask import flash
from flask import g
from flask import redirect
from flask import render_template
from flask import request
from flask import session
from flask import url_for
from werkzeug.security import check_password_hash
from werkzeug.security import generate_password_hash

In [ ]:
#export
bp = Blueprint("auth", __name__, url_prefix="/auth")

def login_required(view):
    """View decorator that redirects anonymous users to the login page."""

    @functools.wraps(view)
    def wrapped_view(**kwargs):
        if g.user is None:
            return redirect(url_for("auth.login"))
        return view(**kwargs)
    return wrapped_view

@bp.before_app_request
def load_logged_in_user():
    """If a user id is stored in the session, load the user object from
    the database into ``g.user``."""
    user_id = session.get("user_id")

    if user_id is None:
        g.user = None
    else:
        g.user = g.service.read_user_by_id(user_id)

In [ ]:
# @bp.route("/register", methods=("GET", "POST"))
# def register():
#     """Register a new user.
#     Validates that the username is not already taken. Hashes the
#     password for security.
#     """
#     if request.method == "POST":
#         username = request.form["username"]
#         password = request.form["password"]
#         db = get_db()
#         error = None

#         if not username:
#             error = "Username is required."
#         elif not password:
#             error = "Password is required."
#         elif (
#             db.execute("SELECT id FROM user WHERE username = ?", (username,)).fetchone()
#             is not None
#         ):
#             error = "User {0} is already registered.".format(username)

#         if error is None:
#             # the name is available, store it in the database and ...
#             db.execute(
#                 "INSERT INTO user (username, password) VALUES (?, ?)",
#                 (username, generate_password_hash(password)),
#             )
#             db.commit()
#             # store the user id in a new session and return to the index
#             session.clear()
#             user = db.execute(
#                 "SELECT * FROM user WHERE username = ?", (username,)
#             ).fetchone()
#             session["user_id"] = user["id"]
#             return redirect(url_for("index"))

#         flash(error)

#     return render_template("auth/register.html")


# @bp.route("/login", methods=("GET", "POST"))
# def login():
#     """Log in a registered user by adding the user id to the session."""
#     print('login' in request.form)
#     print(request.method)
#     print(request.form)
#     if request.method == "POST":
#         username = request.form["username"]
#         password = request.form["password"]
#         db = get_db()
#         error = None
#         user = db.execute(
#             "SELECT * FROM user WHERE username = ?", (username,)
#         ).fetchone()

#         if user is None:
#             error = "Incorrect username."
#         elif not check_password_hash(user["password"], password):
#             error = "Incorrect password."

#         if error is None:
#             # store the user id in a new session and return to the index
#             session.clear()
#             session["user_id"] = user["id"]
#             return redirect(url_for("index"))

#         flash(error)

#     return render_template("auth/login.html")

In [ ]:
#export
@bp.route("/login", methods=("GET", "POST"))
def login():
    """Log in a registered user by adding the user id to the session."""
    if request.method == "POST":
        username = request.form["username"]
        password = request.form["password"]
        error = None
        user = g.service.read_user_by_username(username)
        
        if not username: error = "Username is required."
        elif not password: error = "Password is required."  
        elif 'login' in request.form:
            if user is None:
                error = "Incorrect username."
            elif not check_password_hash(user["password"], password):
                error = "Incorrect password."
        elif 'register' in request.form:
            if user: 
                error = f"User {username} is already registered."
        else:
            error = "Unknown action: we only know about login and register"

        if error is None:
            if 'register' in request.form:
                # the name is available, store it in the database and read the new user row
                g.service.create_user(username, generate_password_hash(password))
                user=g.service.read_user_by_username(username)
            # store the user id in a new session and return to the index
            session.clear()
            session["user_id"] = user["id"]
            return redirect(url_for("index"))

        flash(error)
        
    return render_template("auth/login.html")

In [ ]:
#export
@bp.route("/logout")
def logout():
    """Clear the current session, including the stored user id."""
    session.clear()
    return redirect(url_for("index"))

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 40a_service_db.ipynb.
Converted 40b_service_filesystem.ipynb.
Converted 50_web_app.ipynb.
Converted 50b_web_auth.ipynb.
Converted 50c_web_blog.ipynb.
Converted index.ipynb.
